# Extract labels from BIDS metadata

Five labels can be extracted depending on the diagnostic status of the participants during the follow-up:
- **CN**: sessions of subjects who were diagnosed as _cognitively normal_ during all their follow-up;
- **AD**: sessions of subjects who were diagnosed as _demented_ during all their follow-up;
- **MCI**: sessions of subjects who were diagnosed as _prodromal_ at baseline, who did not encounter multiple reversions and conversions and who did not convert back to cognitively normal status;
- **pMCI**: sessions of subjects who were diagnosed as _prodromal_ at baseline, and _progressed to dementia_ during the <font color='red'> 36 months (time horizon)</font> following the current visit;
- **sMCI**: sessions of subjects who were diagnosed as _prodromal_ at baseline, _remained stable_ during the <font color='red'> 36 months (time horizon)</font> following the current visit and _never progressed to dementia_.

<div class="alert alert-block alert-info">
<b>Time horizon:</b><p>
    In this notebook, we set the time horizon to 36 months.
    The time horizon allows to study the MCI patients stability and to sort them between sMCI and pMCI classes.</p>
    <img src="./images/MCI_stability.png">
</div>


## Merge information with `clinica iotools`

BIDS information is dispatched between different tsv depending if it concerns a subject, a session or a scan.
To gather all data included in tsv files of the BIDS, use the command:

In [ ]:
!clinica iotools merge-tsv <bids_directory> data/test_BIDS.tsv

Another command is also needed to identify which modalities are present for each subject. This will allow to restrict our sessions list to sessions comprising T1-MRI.

This is the aim of the following command: 

In [ ]:
!clinica iotools check-missing-modalities <bids_directory> data/test_missing_mods

## Extract labels with `clinicadl tsvtool` on OASIS

In this section we will now use the OASIS dataset on which `clinica iotools merge-tsv` and `clinica iotools check-missing-modalities` was already performed.

The outputs of the corresponding pipelines can be found respectively at `data/OASIS_BIDS.tsv` and `data/OASIS_missing_mods`.

In [ ]:
ls data

### Restrictions on OASIS dataset

In OASIS, the age distribution of CN participants is different from the one of AD participants. This is an issue as the classifier may learn a difference between age classes instead of learning atrophy patterns caused by Alzheimer's disease on such dataset.

<font color='green'>Should we add here a table or a piece of code to show the difference between the classes ? 
Also maybe we should emphasize the fact that age is also related to atrophy so it is why it is so important to have similar distributions ?</font>

To avoid this bias, CN participants younger than the youngest AD patient were removed. This restriction can be performed with the following commandline:

In [ ]:
!clinicadl tsvtool restrict OASIS data/OASIS_BIDS.tsv data/OASIS_restricted_BIDS.tsv

In [ ]:
ls data

Some other sessions were also excluded because the preprocessing operations failed. The list of the images that were kept after the preprocessing is stored at `data/OASIS_qc_output.tsv`.

### Get the labels

The OASIS dataset only includes **AD** and **CN** labels. They can be extracted in a new folder `labels_lists` from the restricted dataset with the following commandline:

In [ ]:
!clinicadl tsvtool getlabels data/OASIS_restricted_BIDS.tsv data/OASIS_missing_mods data/labels_lists --restriction_path data/OASIS_after_qc.tsv

In [ ]:
ls data/labels_lists

For each diagnosis label, one separate file has been created with all the sessions that can be included in the classification task.

### Demographic analysis

The age bias in OASIS is well known and this is why the youngest CN participants were excluded in a previous section. However, other biases may exist, in particular after the preprocessing step which removed sessions from the dataset. Thus it is crucial to check before going further that there are no other biases in the dataset.

The following command will extract statistical values on the populations for each diagnosis label. Based on those it is possible to check that the dataset is suitable for the classification task.

In [ ]:
!clinicadl tsvtool analysis data/OASIS_BIDS.tsv data/labels_lists data/OASIS_analysis.tsv

In [ ]:
def display_table(table_path):
    import pandas as pd

    OASIS_analysis_df = pd.read_csv(table_path, sep='\t')
    OASIS_analysis_df.set_index("diagnosis", drop=True, inplace=True)
    columns = ["n_subjects", "n_scans",
               "mean_age", "std_age", "min_age", "max_age",
               "sexF", "sexM",
               "mean_MMSE", "std_MMSE", "min_MMSE", "max_MMSE",
               "CDR_0", "CDR_0.5", "CDR_1", "CDR_2", "CDR_3"]

    # Print formatted table
    format_columns = ["subjects", "scans", "age", "sex", "MMSE", "CDR"]
    format_df = pd.DataFrame(index=OASIS_analysis_df.index, columns=format_columns)
    for idx in OASIS_analysis_df.index.values:    
        row_str = "%i; %i; %.1f ± %.1f [%.1f, %.1f]; %iF / %iM; %.1f ± %.1f [%.1f, %.1f]; 0: %i, 0.5: %i, 1: %i, 2:%i, 3:%i" % tuple([OASIS_analysis_df.loc[idx, col] for col in columns])
        row_list = row_str.split(';')
        format_df.loc[idx] = row_list

    format_df.index.name = None
    display(format_df)

In [ ]:
display_table("data/OASIS_analysis.tsv")

There is no significant bias on age anymore, but do you notice any other problems ? What kind of procedure could be done a posteriori to check that no bias was learnt ?

<font color='green'>What do we say about the sex differences in OASIS ?</font>

# Perform data splits

Now that the labels have been extracted and possible biases have been identified, data can be split in different sets. This step is essential to guarantee the independence of the final evaluation.

<div class="alert alert-block alert-info">
<b>Definition of sets:</b><p>
    In this notebook, data is divided between train, validation and test sets:
<ul>
    <li> The <b>train set</b> is used to update the weights, </li>
    <li> The <b>validation set</b> is used to stop the training process and select the best model, </li>
    <li> The <b>test set</b> is used after the end of the training process to perform an unbiased evaluation of the performance. </li>
</ul>
    <img src="./images/split.png">
    <p>Due to the k-fold validation procedure, k trainings are conducted according to the k training/validation pairs generated. This leads to k different models that are evaluated on the test set at the end. The final test performance is then the mean value of these k models.</p>
</div>

## Build test set

The test set is obtained by performing a split on each diagnosis label independently. Each tsv file is split until there is non-significant differences between age and sex distributions between the test set and the train + validation set. Three tsv files are written per label based on sets:

- the baseline sessions of the test set
- the baseline sessions of the train + validation set
- the longitudinal sessions of the train + validation set

In OASIS there is no longitudinal follow-up, hence the two last tsv files are identical.

Let's create a test sets including 20 subjects per label:

In [ ]:
!clinicadl tsvtool split data/OASIS_BIDS.tsv data/labels_lists --n_test 20

The differences between populations of the train + validation and test sets can be assessed to check that there is no discrepancies between the two sets.


In [ ]:
!clinicadl tsvtool analysis data/OASIS_BIDS.tsv data/labels_lists/train data/OASIS_trainval_analysis.tsv

In [ ]:
!clinicadl tsvtool analysis data/OASIS_BIDS.tsv data/labels_lists/test data/OASIS_test_analysis.tsv --baseline

In [ ]:
print("Train + validation set")
display_table("data/OASIS_trainval_analysis.tsv")
print("Test set")
display_table("data/OASIS_test_analysis.tsv")

If you are not satisfied with these populations, you can relaunch the test or change the parameters used to evaluate the difference between the distributions: `p_val_threshold` and `t_val_threshold`.

## Build validation sets

To better estimate the performance of the network, it is trained 5 times using a 5-fold cross-validation procedure. In this procedure, each sample is used once to validate and the other times to train the network.

In a similar way than for the test split, three tsv files are written per label **per split** for each set:

- the baseline sessions of the validation set
- the baseline sessions of the train set
- the longitudinal sessions of the train set

Contrary to the test split, there is no attempt to control the similarity between the age and sex distributions.

In [ ]:
!clinicadl tsvtool kfold data/labels_lists/train --n_splits 5

In [ ]:
ls data/labels_list/train

## Check the absence of data leakage

In OASIS-1 there is no risk of data leakage due to the data split itself as there is only one session per subject. Also there is no MCI patients, hence there is no risk of data leakage during a transfer learning between a source task implying the MCI set and a target task iplying at least one of its subsets (sMCI or pMCI).

One script in the clinicadl library has been created to check that there was no data leakage after the split step. More specifically it checks that:

1. baseline datasets contain only one scan per subject,
2. no intersection between train and test sets,
3. the absence of MCI train subjects in test sets of subcategories of MCI.
    
As it is not a common function, it has not been integrated to the general commandline.

In [ ]:
from clinicadl.tools.tsv.test import run_test_suite

# Run check for train+val / test split
run_test_suite("./data/labels_lists", n_splits=0, subset_name="test")

# Run check for train / validation splits
run_test_suite("./data/labels_lists/train", n_splits=5, subset_name="validation")

If no Error was raised then none of the three conditions was broken. It is now possible to use the train and the validation sets to perform a classification task, and then to evaluate correctly the performance of the classifier on the test set.